Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [47]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [48]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [49]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [50]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

## L2 loss in the weights

In [20]:
batch_size = 128
import numpy
hidden_layer_size = 1024

# multinomial logistic regression 
train_subset = 10000
beta = 0.01

def logits_two_layer(input_x, weights_1, biases_1, weights_2, biases_2):
  hidden_scores = tf.matmul(input_x, weights_1) + biases_1
  relu_hidden = tf.nn.relu(hidden_scores)
  logits = tf.matmul(relu_hidden, weights_2) + biases_2
  return logits  
    
graph_two_layer = tf.Graph()
with graph_two_layer.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_layer_size]))
  biases_1 = tf.Variable(tf.zeros([hidden_layer_size]))

  weights_2 = tf.Variable(
    tf.truncated_normal([hidden_layer_size, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = logits_two_layer(tf_train_dataset, weights_1, biases_1, weights_2, biases_2)  
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  loss = tf.reduce_mean(loss + beta * tf.nn.l2_loss(weights_1) )
  loss = tf.reduce_mean(loss + beta * tf.nn.l2_loss(weights_2) )
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(logits_two_layer(tf_valid_dataset, weights_1, biases_1, weights_2, biases_2))
  test_prediction = tf.nn.softmax(logits_two_layer(tf_test_dataset, weights_1, biases_1, weights_2, biases_2))

In [21]:
num_steps = 3001

with tf.Session(graph=graph_two_layer) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 3506.412598
Minibatch accuracy: 8.6%
Validation accuracy: 22.7%
Minibatch loss at step 500: 21.401011
Minibatch accuracy: 79.7%
Validation accuracy: 84.0%
Minibatch loss at step 1000: 1.048753
Minibatch accuracy: 80.5%
Validation accuracy: 84.5%
Minibatch loss at step 1500: 0.717729
Minibatch accuracy: 84.4%
Validation accuracy: 84.7%
Minibatch loss at step 2000: 0.783058
Minibatch accuracy: 80.5%
Validation accuracy: 84.3%
Minibatch loss at step 2500: 0.791588
Minibatch accuracy: 81.2%
Validation accuracy: 84.7%
Minibatch loss at step 3000: 0.758934
Minibatch accuracy: 80.5%
Validation accuracy: 84.2%
Test accuracy: 89.4%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [27]:
num_steps = 3001
small_size = 400
train_dataset_small = train_dataset[:small_size, :]
train_labels_small = train_labels[:small_size]

# train_dataset_small = train_dataset
# train_labels_small = train_labels

with tf.Session(graph=graph_two_layer) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels_small.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset_small[offset:(offset + batch_size), :]
    batch_labels = train_labels_small[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 3470.046631
Minibatch accuracy: 12.5%
Validation accuracy: 29.1%
Minibatch loss at step 500: 21.114208
Minibatch accuracy: 100.0%
Validation accuracy: 76.5%
Minibatch loss at step 1000: 0.408675
Minibatch accuracy: 100.0%
Validation accuracy: 78.3%
Minibatch loss at step 1500: 0.268074
Minibatch accuracy: 100.0%
Validation accuracy: 78.5%
Minibatch loss at step 2000: 0.251235
Minibatch accuracy: 100.0%
Validation accuracy: 78.4%
Minibatch loss at step 2500: 0.240550
Minibatch accuracy: 100.0%
Validation accuracy: 78.1%
Minibatch loss at step 3000: 0.230576
Minibatch accuracy: 100.0%
Validation accuracy: 78.5%
Test accuracy: 85.0%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

## Two layer network with Dropout

In [45]:
batch_size = 128
import numpy
hidden_layer_size = 1024

# multinomial logistic regression 
train_subset = 10000
beta = 0.01

def logits_two_layer(input_x, weights_1, biases_1, weights_2, biases_2, dropout):
  hidden_scores = tf.matmul(input_x, weights_1) + biases_1
  relu_hidden = tf.nn.relu(hidden_scores)
  if dropout != 0.0:
      relu_hidden = tf.nn.dropout(relu_hidden, dropout)    
  logits = tf.matmul(relu_hidden, weights_2) + biases_2
  return logits  
    
graph_two_layer = tf.Graph()
with graph_two_layer.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  dropout_prob = tf.placeholder("float")
  # Variables.
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_layer_size]))
  biases_1 = tf.Variable(tf.zeros([hidden_layer_size]))

  weights_2 = tf.Variable(
    tf.truncated_normal([hidden_layer_size, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = logits_two_layer(tf_train_dataset, weights_1, biases_1, weights_2, biases_2, dropout_prob)  
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  loss = tf.reduce_mean(loss + beta * tf.nn.l2_loss(weights_1) )
  loss = tf.reduce_mean(loss + beta * tf.nn.l2_loss(weights_2) )
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(logits_two_layer(tf_valid_dataset, weights_1, biases_1, weights_2, biases_2, 0.0))
  test_prediction = tf.nn.softmax(logits_two_layer(tf_test_dataset, weights_1, biases_1, weights_2, biases_2, 0.0))

In [46]:
num_steps = 3001

with tf.Session(graph=graph_two_layer) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,  dropout_prob:0.6}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 3556.130859
Minibatch accuracy: 10.2%
Validation accuracy: 25.6%
Minibatch loss at step 500: 21.391180
Minibatch accuracy: 78.9%
Validation accuracy: 84.0%
Minibatch loss at step 1000: 1.148330
Minibatch accuracy: 75.0%
Validation accuracy: 84.3%
Minibatch loss at step 1500: 0.780546
Minibatch accuracy: 84.4%
Validation accuracy: 84.3%
Minibatch loss at step 2000: 0.824726
Minibatch accuracy: 80.5%
Validation accuracy: 84.1%
Minibatch loss at step 2500: 0.838664
Minibatch accuracy: 81.2%
Validation accuracy: 84.2%
Minibatch loss at step 3000: 0.806342
Minibatch accuracy: 78.9%
Validation accuracy: 84.0%
Test accuracy: 89.6%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


### Four layer Network

In [135]:
batch_size = 128
import numpy
hidden_layer_sizes = [512, 512]

beta = 0.01

def logits_two_layer(input_x, weights_1, biases_1, weights_2, biases_2, weights_3, biases_3, 
                     dropout):
  hidden_scores = tf.matmul(input_x, weights_1) + biases_1
  relu_hidden = tf.nn.relu(hidden_scores)
  if dropout != 0.0:
      relu_hidden = tf.nn.dropout(relu_hidden, dropout)    
        
  hidden_scores_1 = tf.matmul(relu_hidden, weights_2) + biases_2
  relu_hidden_1 = tf.nn.relu(hidden_scores_1)
  if dropout != 0.0:
      relu_hidden_1 = tf.nn.dropout(relu_hidden_1, dropout)    

  logits = tf.matmul(relu_hidden_1, weights_3) + biases_3
        
  return logits  
    
graph_two_layer = tf.Graph()
with graph_two_layer.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  dropout_prob = tf.placeholder("float")
  # Variables.
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_layer_sizes[0]]))
  biases_1 = tf.Variable(tf.zeros([hidden_layer_sizes[0]]))

  weights_2 = tf.Variable(
    tf.truncated_normal([hidden_layer_sizes[0], hidden_layer_sizes[1]]))
  biases_2 = tf.Variable(tf.zeros([hidden_layer_sizes[1]]))
    
  weights_3 = tf.Variable(
    tf.truncated_normal([hidden_layer_sizes[1], num_labels]))
  biases_3 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = logits_two_layer(tf_train_dataset, weights_1, biases_1, weights_2, biases_2, 
                            weights_3, biases_3, dropout_prob)  
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  loss = tf.reduce_mean(loss + beta * tf.nn.l2_loss(weights_1) )
  loss = tf.reduce_mean(loss + beta * tf.nn.l2_loss(weights_2) )
  loss = tf.reduce_mean(loss + beta * tf.nn.l2_loss(weights_3) )

  global_step = tf.Variable(0)  # count the number of steps taken.
  learnr = tf.placeholder("float")
  learning_rate = tf.train.exponential_decay(learnr, global_step, 1000, 0.95, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step= global_step)

  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(logits_two_layer(tf_valid_dataset, weights_1, biases_1, weights_2, biases_2,  weights_3, biases_3, 0.0))
  test_prediction = tf.nn.softmax(logits_two_layer(tf_test_dataset, weights_1, biases_1, weights_2, biases_2,  weights_3, biases_3, 0.0))

In [ ]:
num_steps = 5001

with tf.Session(graph=graph_two_layer) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, 
                 dropout_prob : 0.7, learnr: 0.001}
    _, l, predictions = session.run( [optimizer, loss, train_prediction], feed_dict=feed_dict )
    if (step % 800 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 8129.401855
Minibatch accuracy: 10.2%
Validation accuracy: 8.7%
Minibatch loss at step 800: 3084.671143
Minibatch accuracy: 60.2%
Validation accuracy: 80.9%
Minibatch loss at step 1600: 2749.383545
Minibatch accuracy: 73.4%
Validation accuracy: 81.5%
Minibatch loss at step 2400: 2607.549072
Minibatch accuracy: 71.9%
Validation accuracy: 81.9%
Minibatch loss at step 3200: 2523.773926
Minibatch accuracy: 72.7%
Validation accuracy: 82.0%
Minibatch loss at step 4000: 2465.458740
Minibatch accuracy: 69.5%
Validation accuracy: 82.0%
Minibatch loss at step 4800: 2413.143799
Minibatch accuracy: 76.6%
